In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import numpy as np 
import sys

sys.path.append('..')
torch.set_grad_enabled(False)

In [13]:
device = "cpu"
model_name = "EleutherAI/pythia-70m-deduped"
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float32).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
MODEL_CONFIG={"n_heads":model.config.num_attention_heads,
                      "n_layers":model.config.num_hidden_layers,
                      "resid_dim": model.config.hidden_size,
                      "name_or_path":model.config.name_or_path,
                      "attn_hook_names":[f'gpt_neox.layers.{layer}.attention.dense' for layer in range(model.config.num_hidden_layers)],
                      "layer_hook_names":[f'gpt_neox.layers.{layer}' for layer in range(model.config.num_hidden_layers)]}

In [2]:
# from utils.prompt_utils import *
# from utils.intervention_utils import *
# from utils.model_utils import *
# from utils.eval_utils import *
# from utils.extract_utils import *
# from compute_indirect_effect import compute_indirect_effect

ModuleNotFoundError: No module named 'utils'

In [ ]:
# dataset = load_dataset(dataset_name, root_data_dir=root_data_dir, test_size=test_split, seed=seed)

In [ ]:
dataset_name='antonym'
n_top_heads=10
edit_layer=-1
model_name='EleutherAI/pythia-70m-deduped'
root_data_dir='../dataset_files'
save_path_root='../results'
ie_path_root=None
seed=42
device='cpu'
mean_activations_path=None
indirect_effect_path=None
test_split=0.3
n_shots=10
n_trials=25
prefixes={
    'input': 'Q:',
    'output': 'A:',
    'instructions': ''}
separators={'input': '\n',
'output': '\n\n',
'instructions': ''}
compute_baseline=True
generate_str=False
metric='f1_score'
universal_set=False

In [5]:
model

GPTNeoXForCausalLM(
  (gpt_neox): GPTNeoXModel(
    (embed_in): Embedding(50304, 512)
    (emb_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-5): 6 x GPTNeoXLayer(
        (input_layernorm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (post_attention_dropout): Dropout(p=0.0, inplace=False)
        (post_mlp_dropout): Dropout(p=0.0, inplace=False)
        (attention): GPTNeoXAttention(
          (rotary_emb): GPTNeoXRotaryEmbedding()
          (query_key_value): Linear(in_features=512, out_features=1536, bias=True)
          (dense): Linear(in_features=512, out_features=512, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (mlp): GPTNeoXMLP(
          (dense_h_to_4h): Linear(in_features=512, out_features=2048, bias=True)
          (dense_4h_to_h): Linear(in_features=2048, out_features=512, bias=True)
          (a

In [17]:
mean_acts = torch.load("../results/antonym_EleutherAI_pythia-70m-deduped/antonym_mean_head_activations.pt")
mean_acts.shape # layer, head, tokens, head_dims

torch.Size([6, 8, 97, 64])

In [24]:
indirect_effect = torch.load("../results/antonym_EleutherAI_pythia-70m-deduped/antonym_indirect_effect.pt")
indirect_effect.shape # n_trials, layer, head

torch.Size([25, 6, 8])

In [25]:
aie = indirect_effect.mean(dim=0)
aie.shape

torch.Size([6, 8])

In [ ]:
h_shape = mean_indirect_effect.shape 
topk_vals, topk_inds  = torch.topk(mean_indirect_effect.view(-1), k=n_top_heads, largest=True)
top_lh = list(zip(*np.unravel_index(topk_inds, h_shape), [round(x.item(),4) for x in topk_vals]))
top_heads = top_lh[:n_top_heads]

In [12]:
model.gpt_neox.layers[3]

GPTNeoXLayer(
  (input_layernorm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
  (post_attention_layernorm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
  (post_attention_dropout): Dropout(p=0.0, inplace=False)
  (post_mlp_dropout): Dropout(p=0.0, inplace=False)
  (attention): GPTNeoXAttention(
    (rotary_emb): GPTNeoXRotaryEmbedding()
    (query_key_value): Linear(in_features=512, out_features=1536, bias=True)
    (dense): Linear(in_features=512, out_features=512, bias=True)
    (attention_dropout): Dropout(p=0.0, inplace=False)
  )
  (mlp): GPTNeoXMLP(
    (dense_h_to_4h): Linear(in_features=512, out_features=2048, bias=True)
    (dense_4h_to_h): Linear(in_features=2048, out_features=512, bias=True)
    (act): GELUActivation()
  )
)

In [22]:
MODEL_CONFIG["resid_dim"], MODEL_CONFIG["n_heads"], len(MODEL_CONFIG["attn_hook_names"])

(512, 8, 6)

Todo:
1. What is 97? 
2. What does 70m model score on this task?
3. Get mean activations for all tasks
4. Do PCA on mean activations? 
    - Doesn't PCA do highest variability? Will the 'task vector subspace' be identified by it?